## Experiments


In [71]:
from dotenv import load_dotenv
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder, PromptTemplate
from langchain.schema.output_parser import StrOutputParser
from langchain.output_parsers import PydanticOutputParser
from langchain_core.pydantic_v1 import BaseModel, Field, validator
from langchain.memory import ConversationBufferMemory
from langchain_openai import ChatOpenAI

In [72]:
load_dotenv()

True

In [73]:
def load_txt(paths: list[str]) -> str:
    """Load the contents of a file and return it as a string"""
    for path in paths:
        with open(path, encoding="utf-8") as f:
            yield f.read()

def extract_variables(prompt: str, opening="{", closing="}") -> list[str]:
    """Extract and return variables encolsed in specific delimeters from the given prompt string."""
    variables = []
    inside_braces = False
    current_variable = []
    
    for char in prompt:
        if char == opening:
            inside_braces = True
            current_variable = []
            
        elif char == closing:
            if current_variable and inside_braces:
                variables.append(''.join(current_variable))
            inside_braces = False
        elif inside_braces:    
            current_variable.append(char)
            
    return variables

In [75]:
# ENum 
from enum import Enum
   
class Dimension(str, Enum):
    UNDERSTANDING = "Understanding of the problem"
    PROBLEM_SOLVING = "Problem-solving approach"
    CODE_QUALITY = "Code efficiency and cleanliness"
    COMMUNICATION = "Communication skills"
    TIME_MANAGEMENT = "Time management"
    
class EvaluationModel(BaseModel):
    """Model for the evaluation of the user's response."""
    dimension: Dimension = Field(description="The dimension being evaluated. One of the following: Understanding of the problem, Problem-solving approach, Code efficiency and cleanliness, Communication skills, Time management.")
    score: float = Field(description="The score of the user's response from 100.")
    feedback: str = Field(description="The feedback for the user's response. Concise and evidence based.")


class InterviewModel(BaseModel):
    end: bool = Field(description="Whether the interview has ended.")
    response: str = Field(description="The response from the interviewer that will be spoken out to the user.")

    evaluation: list[EvaluationModel] = Field(description="List of evaluations. This should be returned only when the interview has ended.", default=[])
    

In [76]:
system_prompt, user_prompt = load_txt(["system_prompt.txt", "user_prompt.txt"])
print(f"Variables:\n\t SYSTEM: {extract_variables(system_prompt)}\n\t USER: {extract_variables(user_prompt)} ")

Variables:
	 SYSTEM: ['problem', 'time_allotted']
	 USER: ['user_input', 'code_snapshot', 'time_elapsed'] 


In [77]:
parser = PydanticOutputParser(pydantic_object=InterviewModel)

interviewer_template = PromptTemplate(
        template= system_prompt + "\n\n Chat History: {chat_history}\n\n" + "\n\n  {format_instructions} \n\n" + user_prompt,
        input_variables=["problem", "time_allotted", "user_input", "code_snapshot", "time_elapsed", "chat_history"],
        partial_variables={"format_instructions": parser.get_format_instructions()},
    )

llm = ChatOpenAI(model="gpt-4o-mini")
memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)

In [78]:
interview_chain = interviewer_template | llm | parser

In [79]:
problem = """
27. Remove Element (Easy)

Given an integer array nums and an integer val, remove all occurrences of val in nums in-place. The order of the elements may be changed. Then return the number of elements in nums which are not equal to val.

Consider the number of elements in nums which are not equal to val be k, to get accepted, you need to do the following things:

Change the array nums such that the first k elements of nums contain the elements which are not equal to val. The remaining elements of nums are not important as well as the size of nums.
Return k.
"""

In [80]:
import time
from IPython.display import display, clear_output
import ipywidgets as widgets

# Provided variables
time_allotted = 40
user_input = None
code_snapshot = None
time_elapsed = None
i = 0

# Provided print statement
print("Welcome to the DSA Interview Simulator!")

# Function to handle interaction in Jupyter Notebook
def handle_interaction(submit_btn):
    global user_input, code_snapshot, time_elapsed, i
    
    # Update the time elapsed
    time_elapsed = i * 10

    # Capture user input and code snapshot from widgets
    user_input = user_input_widget.value
    code_snapshot = code_snapshot_widget.value

    # Prepare the variables dictionary as provided
    variables = {
        "problem": problem,
        "time_allotted": time_allotted,
        "user_input": user_input,
        "code_snapshot": code_snapshot,
        "time_elapsed": time_elapsed,
        "chat_history": memory.buffer_as_messages
    }

    # Invoke the AI chain with the provided variables
    ai = interview_chain.invoke(variables)
    
    # Display AI response
    clear_output(wait=True)
    print("Welcome to the DSA Interview Simulator!")
    print("AI:", ai)
    
    # Update widgets for the next input
    user_input_widget.value = ''
    code_snapshot_widget.value = ''

    # Simulate the memory updates and user input handling
    memory.chat_memory.add_ai_message(str(ai))
    memory.chat_memory.add_user_message(
        user_prompt_template.invoke({
            "user_input": user_input,
            "code_snapshot": code_snapshot,
            "time_elapsed": time_elapsed
        }).to_string()
    )
    
    # Increment the iteration counter
    i += 1
    
    # Display the interaction box again for further inputs
    display(interaction_box)

# Widgets for user input and code snapshot
user_input_widget = widgets.Textarea(
    value='',
    placeholder='Enter your message...',
    description='Message:',
    layout=widgets.Layout(width='100%', height='100px')
)

code_snapshot_widget = widgets.Textarea(
    value='',
    placeholder='Paste your code here...',
    description='Code:',
    layout=widgets.Layout(width='100%', height='200px')
)

# Submit button
submit_button = widgets.Button(
    description='Submit',
    button_style='primary',
    layout=widgets.Layout(width='100%')
)

# Attach the button click event to handle_interaction
submit_button.on_click(handle_interaction)

# Box layout for the interaction UI
interaction_box = widgets.VBox([user_input_widget, code_snapshot_widget, submit_button])

# Initial UI display
clear_output(wait=True)
print("Welcome to the DSA Interview Simulator!")

display(interaction_box)

Welcome to the DSA Interview Simulator!
AI: end=True response="Thank you for your time, Munim! We have reached the end of our interview. Let's go over your performance. You demonstrated a good understanding of the problem requirements and outlined a clear approach to solving it. Your method of using a counter and iterating through the array was solid. However, you could enhance your approach by detailing how to efficiently shift elements in place. Your communication was clear throughout the interview, but make sure to keep your answers concise. Overall, I would rate your understanding of the problem at eighty-five out of one hundred. Your problem-solving approach also scores eighty-five, while your code efficiency and cleanliness could be improved, so I would give that a seventy-five. Communication skills were strong, scoring ninety, and your time management was fair, scoring seventy. Keep practicing, and I wish you the best in your future endeavors!" evaluation=[EvaluationModel(dimens